# Use word2vec (if-idf for centroid) and random forest to tagging the question.

[Ref: https://www.kaggle.com/code/antonsruberts/sentence-embeddings-centorid-method-vs-doc2vec]

(It not effective, but it's a good practice for me to use word2vec and random forest)

```
Hamming Loss:  0.01
F1 Score:  0.03015075376884422
```

In [3]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import mysql.connector
import json

# Establish a connection to the MySQL database
connection = mysql.connector.connect(
    host='127.0.0.1',
    port=13306,
    user='root',
    password='root',
    database='pyml'
)

# Read the table data using pandas
query = """SELECT vi.hash_id, vi.contents as question_content, JSON_UNQUOTE(JSON_EXTRACT(tags, '$[*].slug')) AS slug FROM viblo_interview vi"""
df_ques = pd.read_sql(query, connection)

query_ans = """SELECT va.hash_id, va.contents, va.question_id FROM viblo_answer va"""
df_answer = pd.read_sql(query_ans, connection)

# Close the database connection
connection.close()


def filter_slug(x):
    if x is None:
        return ''
    t = str(x).replace("b'", "").replace("'", "")
    t1 = json.loads(t)

    return t1
df_ques['slug_filter'] = df_ques['slug'].apply(filter_slug)
print(df_ques['slug_filter'][0])


/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_1577/1993668918.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ques = pd.read_sql(query, connection)
/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_1577/1993668918.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_answer = pd.read_sql(query_ans, connection)


['javascript']


In [4]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

Y = df_ques['slug_filter']
print(Y[100:110])

mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(Y)


print('y_test', Y[100])
t = mlb.inverse_transform(Y[100:101])
print('t', t)
print('class', mlb.classes_)
   
X = df_ques['question_content']
print(len(X))

100          [java]
101         [mysql]
102          [java]
103       [general]
104          [java]
105       [general]
106       [general]
107          [java]
108    [javascript]
109    [javascript]
Name: slug_filter, dtype: object
y_test [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
t [('java',)]
class ['agile' 'ai' 'amazon-web-services' 'android-os' 'angularjs'
 'artificial-intelligence' 'asp-net' 'attack-techniques' 'aws-lambda'
 'backend-development' 'big-data' 'business-analyst' 'c' 'c-2' 'c-lang'
 'chatbot' 'clean-code' 'cloud-computing' 'cloud-security'
 'computer-network' 'computer-vision' 'content-creator' 'cryptography'
 'css' 'cyber-security' 'dart' 'data-science'
 'data-structures-and-algorithms' 'database' 'deep-learning'
 'design-pattern' 'devops' 'distribute-system' 'docker'
 'external-communicati

## Build word2vec for sentences

In [5]:
from underthesea import word_tokenize

def segmentation(text):
    try:
        return word_tokenize(text, format="text")
    except (TypeError, AttributeError):
        return ''

def split_words(post):
    texts = segmentation(post)
    try:
        t = [str(x.strip('0123456789%@$.,=+-!;/()*"&^:#|\n\t\' ').lower()) for x in texts.split()]
        filtered_list = [item for item in t if item != ""]
        return filtered_list
    except TypeError:
        return []
    
X_1 = X.apply(split_words)

In [6]:
print(X_1[:2])

0    [thế, nào, là, abstract, syntax, tree, ?, so, ...
1    [nếu, các, kỹ_thuật, thường, được, sử_dụng, để...
Name: question_content, dtype: object


In [7]:

sentences = X_1

In [8]:
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer

#To proprely work with scikit's vectorizer
merged_questions = [' '.join(question) for question in sentences]
document_names = ['Doc {:d}'.format(i) for i in range(len(merged_questions))]

def get_tfidf(docs, ngram_range=(1,1), index=None):
    vect = TfidfVectorizer(ngram_range=ngram_range)
    tfidf = vect.fit_transform(docs).todense()
    return pd.DataFrame(tfidf, columns=vect.get_feature_names_out(), index=index).T

tfidf = get_tfidf(merged_questions, ngram_range=(1,1), index=document_names)

In [9]:
# Build the word2vec model
n = 100
model = Word2Vec(X_1.values.tolist(), vector_size=n, min_count=1, window = 8)

In [10]:
# Convert the dictionary to a list of key-value pairs and slice the list
first_five = list(model.wv.key_to_index.items())[20:35]

# Print the first 5 elements
print(first_five)

[('khác', 20), ('cách', 21), ('như', 22), ('khi', 23), ('cho', 24), ('có_thể', 25), ('với', 26), ('nhau', 27), ('những', 28), ('giải_thích', 29), ('nêu', 30), ('`', 31), ('khác_biệt', 32), ('tại_sao', 33), ('hãy', 34)]


In [11]:
import numpy as np

def get_sent_embs(emb_model, sentences = [], n = 50):
    sent_embs = []
    for desc in range(len(sentences)):
        sent_emb = np.zeros((1, n))
        if len(sentences[desc]) > 0:
            sent_emb = np.zeros((1, n))
            div = 0
            model = emb_model
            for word in sentences[desc]:
                if word in model.wv.key_to_index and word in tfidf.index:
                    word_emb = model.wv[word]
                    weight = tfidf.loc[word, 'Doc {:d}'.format(desc)]
                    sent_emb = np.add(sent_emb, word_emb * weight)
                    div += weight
                else:
                    div += 1e-13 #to avoid dividing by 0
        if div == 0:
            print(desc)

        sent_emb = np.divide(sent_emb, div)
        sent_embs.append(sent_emb.flatten())
    return sent_embs

ft_sent = get_sent_embs(emb_model = model, sentences=sentences, n = n)

In [12]:
print(len(ft_sent[1]))

100


In [13]:
print(tfidf.loc)

In [14]:
def encode(texts = []):
    embs = get_sent_embs(emb_model = model, sentences=texts, n = 100)
    return embs

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split

X_list = ft_sent
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_list, Y, test_size=0.2, random_state=42)

In [16]:
print(len(X_train), len(X_train[0]), len(Y_train), len(Y_train[0]))

2893 100 2893 109


In [17]:
from sklearn.svm import SVC
# Create a multi-output classifier with a random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
# svc = SVC(kernel='rbf')
clf = MultiOutputClassifier(rf, n_jobs=-1)

# Fit the classifier to the training data
clf.fit(X_train, Y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42),
                      n_jobs=-1)

In [18]:
from sklearn.metrics import hamming_loss, f1_score

prediction = clf.predict(X_test)
# print('Prediction: ', prediction[4:12])
print('Hamming Loss: ', round(hamming_loss(Y_test, prediction),2))
print('F1 Score: ', f1_score(Y_test, prediction, average='micro'))

print('F1 Score: ', f1_score(Y_test, prediction, average=None))

Hamming Loss:  0.01
F1 Score:  0.025284450063211124
F1 Score:  [0.         0.         0.         0.         0.         0.05128205
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.08510638 0.         0.         0.23529412 0.125      0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.5        0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.18181818 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        

/Users/ngocp/.pyenv/versions/3.10.3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [19]:
X_test_free = [
    'Phân biệt mạng Object Detection 2 stage và 1 stage',
    'Những lỗ hổng thường gặp với API',
    'IAM là gì, IAM có vai trò gì trong việc đảm bảo an toàn hệ thống AWS',
    'Có bao nhiêu loại lỗi sai trong PHP?',
    'Git Submodule trong trường hợp nào ?',
    'Có sự thừa kế theo cấp bậc giữa các module trong không? Giải thích.',
    'Sự khác biệt giữa từ khóa break và continue trong Java?',
    'Trình bày về Output Buffering trong PHP?',
    'RESTful API là gì?',
    'Viết tắt của php có nghĩa là gì ?',
    'Phân biệt POST và GET trong php?',
    'Cờ HttpOnly có tác dụng gì cho cookie?',
    'so sánh sự khác nhau giữa Mysql và MongoDB',
    'Tại sao phải sử dụng hàm khởi tạo?',
    'Hash trong Ruby có thể lưu trữ các loại dữ liệu nào?',
]


X_test_embed = encode(X_test_free)

# Predict the labels for the test data
Y_pred_free = clf.predict(X_test_embed)

Y_pred_free_class = mlb.inverse_transform(Y_pred_free)
print('leng : Y_pred_free_class', len(Y_pred_free_class))
for index, tags in enumerate(Y_pred_free_class):
    print(X_test_free[index][:100], '->', tags)

leng : Y_pred_free_class 15
Phân biệt mạng Object Detection 2 stage và 1 stage -> ('mysql', 'programming')
Những lỗ hổng thường gặp với API -> ('mysql', 'programming')
IAM là gì, IAM có vai trò gì trong việc đảm bảo an toàn hệ thống AWS -> ('mysql', 'programming')
Có bao nhiêu loại lỗi sai trong PHP? -> ('mysql', 'programming')
Git Submodule trong trường hợp nào ? -> ('mysql', 'programming')
Có sự thừa kế theo cấp bậc giữa các module trong không? Giải thích. -> ('mysql', 'programming')
Sự khác biệt giữa từ khóa break và continue trong Java? -> ('mysql', 'programming')
Trình bày về Output Buffering trong PHP? -> ('mysql', 'programming')
RESTful API là gì? -> ('mysql', 'programming')
Viết tắt của php có nghĩa là gì ? -> ('mysql', 'programming')
Phân biệt POST và GET trong php? -> ('mysql', 'programming')
Cờ HttpOnly có tác dụng gì cho cookie? -> ('mysql', 'programming')
so sánh sự khác nhau giữa Mysql và MongoDB -> ('mysql', 'programming')
Tại sao phải sử dụng hàm khởi tạo? -> ('mysql', 